In [1]:
import os
import pandas as pd
import cv2
import numpy as np
import matplotlib.pyplot as plt
from dataset_tools import *

In [14]:
def generate_ties(dataset,out_dir,length,tie_size,min_r=0,min_roi = 0.5):
    for in_dir,out_dir in iterate_folders(dataset,out_dir):
        iter = 0
        motion_ties = None
        for batch_num,(imgs,masks) in enumerate(iterate_bathced(in_dir,length)):
            print in_dir+' %d %d\r'%(batch_num,iter),
            h,w = tie_size,tie_size
            cols = imgs.shape[2]//w
            rows = imgs.shape[1]//h
            imgs = imgs[:,:h*rows,:w*cols,:]
            masks = masks[:,:h*rows,:w*cols]
            imgs = np.transpose(imgs.reshape((-1,rows,h,cols,w,3)),(1,3,0,2,4,5)).reshape(cols*rows,len(imgs),h,w,3)
            masks = np.transpose(masks.reshape((-1,rows,h,cols,w)),(1,3,0,2,4)).reshape(cols*rows,len(masks),h,w)
            
            masks_mean_motion = np.zeros_like(masks)
            masks_mean_motion[masks > 240] = 1
            masks_mean_motion = masks_mean_motion.mean((2,3))
            
            masks_mean_roi = np.zeros_like(masks)
            masks_mean_roi[(masks > 240)|(masks < 30)] = 1
            masks_mean_roi = masks_mean_roi.mean((2,3))
            
            for i in range(len(masks)):
                m_r = masks_mean_motion[i].mean() 
                m_roi = masks_mean_roi[i].mean()
                if(m_r >= min_r and m_roi >= min_roi):
                    im = ties_to_image(imgs[i])
                    mask = ties_to_image(masks[i])
                    cv2.imwrite(out_dir+'/%d_input.jpg'%(iter),im)
                    cv2.imwrite(out_dir+'/%d_mask.png'%(iter),mask)
                    iter=iter+1

            m_ties = imgs.reshape((-1,h,w,3))[masks_mean_motion.flatten()>=.97]
            if(len(m_ties) > 0):
                if not(motion_ties is None):
                    motion_ties = np.concatenate((motion_ties,m_ties))
                else:
                    motion_ties = m_ties
        if not (motion_ties is None):
            for i in range(int(np.ceil(float(len(motion_ties))/50.))):
                im = ties_to_image(motion_ties[i*50:min((i+1)*50,len(motion_ties))],5,10)
                cv2.imwrite(out_dir+'/%d_motion.jpg'%(i),im)
            print in_dir+' %d ties=%d motion=%d'%(batch_num,iter,len(motion_ties))
        else:
            print in_dir+' %d ties=%d motion=%d'%(batch_num,iter,0)


In [13]:
generate_ties('../gmm_segmentation/dataset','train_ties',256,32,0.01,0.95)

../gmm_segmentation/dataset/baseline/pedestrians 3 ties=64 motion=0
../gmm_segmentation/dataset/baseline/PETS2006 3 ties=238 motion=432
../gmm_segmentation/dataset/cameraJitter/boulevard 8 ties=144 motion=1470
../gmm_segmentation/dataset/cameraJitter/sidewalk 3 ties=0 motion=0
../gmm_segmentation/dataset/cameraJitter/traffic 5 ties=76 motion=1260
../gmm_segmentation/dataset/dynamicBackground/canoe 3 ties=0 motion=0
../gmm_segmentation/dataset/dynamicBackground/fall 14 ties=690 motion=11867
../gmm_segmentation/dataset/dynamicBackground/fountain01 3 ties=5 motion=0
../gmm_segmentation/dataset/dynamicBackground/fountain02 4 ties=31 motion=0
../gmm_segmentation/dataset/dynamicBackground/overpass 10 ties=28 motion=294
../gmm_segmentation/dataset/intermittentObjectMotion/parking 8 ties=0 motion=0
../gmm_segmentation/dataset/intermittentObjectMotion/sofa 9 ties=190 motion=46
../gmm_segmentation/dataset/intermittentObjectMotion/streetLight 11 ties=0 motion=0
../gmm_segmentation/dataset/intermi

In [15]:
generate_ties('../gmm_segmentation/test_dataset','test_ties',256,32,0.01,0.95)

../gmm_segmentation/test_dataset/badWeather/blizzard 26 ties=591 motion=3927
../gmm_segmentation/test_dataset/baseline/highway 5 ties=64 motion=559
../gmm_segmentation/test_dataset/cameraJitter/badminton 3 ties=0 motion=220
../gmm_segmentation/test_dataset/dynamicBackground/boats 30 ties=6 motion=0
../gmm_segmentation/test_dataset/intermittentObjectMotion/abandonedBox 16 ties=1 motion=0
../gmm_segmentation/test_dataset/lowFramerate/port_0_17fps 10 ties=2 motion=0
../gmm_segmentation/test_dataset/nightVideos/bridgeEntry 8 ties=11 motion=0
../gmm_segmentation/test_dataset/PTZ/continuousPan 5 ties=171 motion=759
../gmm_segmentation/test_dataset/shadow/backdoor 6 ties=78 motion=37
../gmm_segmentation/test_dataset/thermal/corridor 20 ties=246 motion=3143
../gmm_segmentation/test_dataset/turbulence/turbulence0 18 ties=103 motion=0
